## AirBnb, New York Local Experience with emphasis on Point of Interest. Kaggle dataset for Airbnb listings and metrics in NYC, NY, USA (2019)


### Import dependencies & configure gmaps with your API key 

In [19]:
# Dependencies and Setup
import requests
import json
import pprint
import pandas as pd
import numpy as np
import time
#from config import gkey
import gmaps
from sqlalchemy import create_engine

Get the NYC data from the API and dataset from Airbnb listing(Kaggle). 

In [20]:
# Pull latest data available as of 2019
airbnb_data_df = pd.read_csv("AB_NYC_2019.csv")

airbnb_data_df.head(20)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,10/19/2018,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,5/21/2019,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,7/5/2019,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,11/19/2018,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,6/22/2019,0.59,1,129
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,10/5/2017,0.40,1,0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,6/24/2019,3.47,1,220
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,7/21/2017,0.99,1,0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,6/9/2019,1.33,4,188


In [23]:
room_type_df = airbnb_data_df[['id', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude','room_type', 'price', 'minimum_nights']].copy()
room_type_df.head(20)

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights
0,2539,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1
1,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1
2,3647,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3
3,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1
4,5022,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10
5,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3
6,5121,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45
7,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2
8,5203,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2
9,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1


In [24]:
private_room_type_df = room_type_df[room_type_df['room_type']=='Private room']
private_room_type_df.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights
0,2539,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1
2,3647,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3
6,5121,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45
7,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2
8,5203,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2


In [26]:
house_room_type_df = room_type_df[room_type_df['room_type']=='Entire home/apt']
house_room_type_df.head()

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights
1,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1
3,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1
4,5022,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10
5,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3
9,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1


In [ ]:
#Connect to local database
pg_user = 'postgres'
pg_password = 'Postgres2019'
db_name = 'AIRbnb_db



In [ ]:
connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#Check for tables
engine.table_names()

In [ ]:
#Use pandas to load csv converted DataFrame into database
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)


In [ ]:
#Confirm data has been added by querying the customer_name table
#NOTE: can also check using pgAdmin
pd.read_sql_query('select * from customer_name', con=engine).head()

In [ ]:
#Confirm data has been added by querying the customer_location table
pd.read_sql_query('select * from customer_location', con=engine).head()

In [27]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [28]:
lat1 = 40.5; lat2 = 42; long1 = -90; long2 = -93
print( distance((lat1, long1), (lat2, long2)) )

301.17000641409464
